# 실습1: Apache 재단에서 관리하는 Service 목록 긁어오기
### 1. By Category 하위에 있는 목록 긁기
### 2. 각각의 목록에 해당하는 URL주소 긁기
#### 참고: https://apache.org/

### 필요한 라이브러리들 import

- 단순히 기계적으로 import하는 것이 아니라, 내가 무엇을 할 것인지 명확하게 할 것.
<br><br>
- 예를들어, Rest API가 아니라 html 태그를 통해 크롤링을 한다면
- 1. 웹페이지에 요청을 보낸다. -> requests 라이브러리가 필요하겠군.
- 2. html 태그를 통해 크롤링을 한다. -> text를 html로 파싱을 해야겠네. -> Beutifulsoup 라이브러리가 필요하겠군.
- 3. 크롤링을 완료하면 하나의 데이터프레임으로 만든 다음 csv 파일로 저장해야지. -> pandas 라이브러리가 필요하겠군.

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
# 데이터를 긁어올 타겟 웹사이트 url 주소
target_url = 'https://www.apache.org/'

# target_url이 가지고 있는 데이터를 get 하겠다.
res = requests.get(target_url)

# 200이 출력되면 성공
res

<Response [200]>

In [3]:
# target_url에서 가져온 데이터는 사실 text에 불과하다.
# 따라서 html로 파싱해주어야 한다.
# 파싱? --- 데이터의 형태를 변환해주는 것이라고 이해하면 편하다.
# (text를 html로 변환한다.)
soup = bs(res.text, 'html.parser')


#### 여기서부터 본격적으로 태그를 통해 크롤링을 하는 과정 ####

# 1. 개발자도구 창 열기 (f12)
# 2. 창 상단에 Elements 클릭
# 3. Elements 왼쪽에 마우스모양 클릭 -> 마우스 모양을 활성화시키고 페이지 내에서 원하는 정보의 위치를 클릭하면 해당 소스코드를 볼 수 있다.
# 4. id는 #으로 접근, class는 .으로 접근
# 5. > 꺽쇠를 사용하면 자식 태그로 들어갈 수 있다.
# 6. 아래 예시 코드를 확인해보자

category_list = soup.select('#by_category > ul > li > a')

# Elements 안에 요소를 우클릭 -> Copy -> Copy selecter 클릭하면 아래와같이 자동으로 태그 level을 형성해주기도 함.
# category_list = soup.select('#by_category > ul:nth-of-type(2) > li > a')


category_list


categories = []
urls = []

for category in category_list:
    categories.append(category.text)

for category_url in category_list:
    
    # html 태그 내의 속성을 가져오고 싶을 때에는 .select가 아니라 .attrs['속성이름']을 사용한다.
    urls.append(category_url.attrs['href'])

tmp_dict = {
    'categories':categories,
    'urls':urls
}

df = pd.DataFrame(tmp_dict)

In [2]:
import os
from google.colab import drive

# 크롤링해서 모은 데이터를 csv파일로 저장한 후 drive에 업로드하는 방법
# 먼저, drive의 경로를 mount해줘야 한다.
# 내가 어디에 저장을 할 것인지 컴퓨터에게 알려주는 작업이다.
drive.mount('/content/drive')

In [44]:
df.to_csv('./drive/MyDrive/test/test.csv', index=False)